In [1]:
import os
from datetime import datetime
import numpy as np
import pandas as pd
import rasterio
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error

In [2]:
# Base folder containing polygon folders
base_folder = r"D:\Yield\LR\SentinelData"

# Polygon folders to process
polygon_folders = ['Munda1', 'Munda2', 'Munda3', 'Munda4', 'Munda5', 'Munda6', 'Munda7']

# Store results
records = []

for poly in polygon_folders:
    folder_path = os.path.join(base_folder, poly)
    files = [f for f in os.listdir(folder_path) if f.endswith('.tif')]
    files.sort()

    for file in files:
        file_path = os.path.join(folder_path, file)
        with rasterio.open(file_path) as src:
            red = src.read(4).astype('float32')
            green = src.read(3).astype('float32')
            nir = src.read(8).astype('float32')

        # Vegetation Indices
        ndvi = (nir - red) / (nir + red + 1e-10)
        gndvi = (nir - green) / (nir + green + 1e-10)
        savi = ((nir - red) / (nir + red + 0.428)) * 1.428  # L=0.428

        # Mask invalid VI values
        ndvi = np.where((ndvi >= -1) & (ndvi <= 1), ndvi, np.nan)
        gndvi = np.where((gndvi >= -1) & (gndvi <= 1), gndvi, np.nan)
        savi = np.where((savi >= -1) & (savi <= 1), savi, np.nan)

        # Calculate VI means
        mean_ndvi = np.nanmean(ndvi)
        mean_gndvi = np.nanmean(gndvi)
        mean_savi = np.nanmean(savi)

        # === NEW: Calculate mean reflectance of raw bands ===
        mean_red = np.nanmean(red)
        mean_green = np.nanmean(green)
        mean_nir = np.nanmean(nir)

        # Extract date
        try:
            date_str = file.split('_')[1]
            date = datetime.strptime(date_str, "%Y%m%d")
        except Exception:
            date = None

        # Append results
        records.append({
            'polygon': poly,
            'date': date,
            'filename': file,
            'mean_NDVI': mean_ndvi,
            'mean_GNDVI': mean_gndvi,
            'mean_SAVI': mean_savi,
            'mean_Red': mean_red,
            'mean_Green': mean_green,
            'mean_NIR': mean_nir
        })

# Create DataFrame
df_indices = pd.DataFrame(records)
print("Raw per-image features:")
print(df_indices.head())


Raw per-image features:
  polygon       date                           filename  mean_NDVI  \
0  Munda1 2024-11-01  M225polygon_20241101_20241201.tif   0.206814   
1  Munda1 2024-12-01  M225polygon_20241201_20241231.tif   0.206242   
2  Munda1 2024-12-31  M225polygon_20241231_20250130.tif   0.185444   
3  Munda1 2025-01-30  M225polygon_20250130_20250301.tif   0.729419   
4  Munda1 2025-03-01  M225polygon_20250301_20250331.tif   0.778799   

   mean_GNDVI  mean_SAVI  mean_Red  mean_Green  mean_NIR  
0    0.371228   0.135172  0.144996    0.100801  0.219772  
1    0.375800   0.136134  0.148264    0.101769  0.224214  
2    0.279439   0.129196  0.167590    0.137188  0.243356  
3    0.660060   0.482791  0.049627    0.065091  0.319557  
4    0.689015   0.544887  0.045231    0.067138  0.365658  


In [3]:
# Load area data
df_area = pd.read_csv("polygon_areas.csv")

# Merge area with vegetation index data
df_indices = pd.merge(df_indices, df_area, on='polygon', how='left')

# Check for any polygons without area info
missing_area = df_indices[df_indices['area_ha'].isna()]
if not missing_area.empty:
    print("Warning: Some polygons are missing area values:")
    print(missing_area['polygon'].unique())

# Show result
print("Merged vegetation index + area data:")
print(df_indices.head())


Merged vegetation index + area data:
  polygon       date                           filename  mean_NDVI  \
0  Munda1 2024-11-01  M225polygon_20241101_20241201.tif   0.206814   
1  Munda1 2024-12-01  M225polygon_20241201_20241231.tif   0.206242   
2  Munda1 2024-12-31  M225polygon_20241231_20250130.tif   0.185444   
3  Munda1 2025-01-30  M225polygon_20250130_20250301.tif   0.729419   
4  Munda1 2025-03-01  M225polygon_20250301_20250331.tif   0.778799   

   mean_GNDVI  mean_SAVI  mean_Red  mean_Green  mean_NIR   area_ha  
0    0.371228   0.135172  0.144996    0.100801  0.219772  0.101974  
1    0.375800   0.136134  0.148264    0.101769  0.224214  0.101974  
2    0.279439   0.129196  0.167590    0.137188  0.243356  0.101974  
3    0.660060   0.482791  0.049627    0.065091  0.319557  0.101974  
4    0.689015   0.544887  0.045231    0.067138  0.365658  0.101974  


In [4]:
df_yield = pd.read_csv("yield_tons.csv")

# Merge with yield data
df = pd.merge(df_indices, df_yield, on='polygon', how='inner')

# Check which columns exist
print("Available columns:", df.columns.tolist())

# Drop rows with missing data including band means
df = df.dropna(subset=[
    'mean_NDVI', 'mean_GNDVI', 'mean_SAVI', 
    'mean_Red', 'mean_Green', 'mean_NIR', 
    'area_ha', 'yield_tons'
])

# Show preview
print("Per-image data ready for modeling:")
print(df.head())


Available columns: ['polygon', 'date', 'filename', 'mean_NDVI', 'mean_GNDVI', 'mean_SAVI', 'mean_Red', 'mean_Green', 'mean_NIR', 'area_ha', 'yield_tons']
Per-image data ready for modeling:
  polygon       date                           filename  mean_NDVI  \
0  Munda1 2024-11-01  M225polygon_20241101_20241201.tif   0.206814   
1  Munda1 2024-12-01  M225polygon_20241201_20241231.tif   0.206242   
2  Munda1 2024-12-31  M225polygon_20241231_20250130.tif   0.185444   
3  Munda1 2025-01-30  M225polygon_20250130_20250301.tif   0.729419   
4  Munda1 2025-03-01  M225polygon_20250301_20250331.tif   0.778799   

   mean_GNDVI  mean_SAVI  mean_Red  mean_Green  mean_NIR   area_ha  yield_tons  
0    0.371228   0.135172  0.144996    0.100801  0.219772  0.101974         0.9  
1    0.375800   0.136134  0.148264    0.101769  0.224214  0.101974         0.9  
2    0.279439   0.129196  0.167590    0.137188  0.243356  0.101974         0.9  
3    0.660060   0.482791  0.049627    0.065091  0.319557  0.101974

In [5]:
# Select only the columns needed for training
df_summary = df[[
    'polygon',
    'mean_NDVI',
    'mean_GNDVI',
    'mean_SAVI',
    'mean_Red',
    'mean_Green',
    'mean_NIR',
    'area_ha',
    'yield_tons',
    'date',
    'filename'
]]

# Drop rows with missing values (optional clean-up)
df_summary = df_summary.dropna()

# Show the final table
print("Image-level training dataset:")
print(df_summary.head())


Image-level training dataset:
  polygon  mean_NDVI  mean_GNDVI  mean_SAVI  mean_Red  mean_Green  mean_NIR  \
0  Munda1   0.206814    0.371228   0.135172  0.144996    0.100801  0.219772   
1  Munda1   0.206242    0.375800   0.136134  0.148264    0.101769  0.224214   
2  Munda1   0.185444    0.279439   0.129196  0.167590    0.137188  0.243356   
3  Munda1   0.729419    0.660060   0.482791  0.049627    0.065091  0.319557   
4  Munda1   0.778799    0.689015   0.544887  0.045231    0.067138  0.365658   

    area_ha  yield_tons       date                           filename  
0  0.101974         0.9 2024-11-01  M225polygon_20241101_20241201.tif  
1  0.101974         0.9 2024-12-01  M225polygon_20241201_20241231.tif  
2  0.101974         0.9 2024-12-31  M225polygon_20241231_20250130.tif  
3  0.101974         0.9 2025-01-30  M225polygon_20250130_20250301.tif  
4  0.101974         0.9 2025-03-01  M225polygon_20250301_20250331.tif  


In [6]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

# Define features to use for training
features = [
    'mean_NDVI',
    'mean_GNDVI',
    'mean_SAVI',
    'mean_Red',
    'mean_Green',
    'mean_NIR'
]

# Drop any rows with missing values in these features (safety check)
df_summary = df_summary.dropna(subset=features + ['yield_tons'])

# Define input (X) and output (y)
X = df_summary[features]
y = df_summary['yield_tons']  # Already in tons/ha

# Train linear regression model
model = LinearRegression()
model.fit(X, y)

# Predict on training data
y_pred = model.predict(X)

# Evaluate model
r2 = r2_score(y, y_pred)
rmse = mean_squared_error(y, y_pred, squared=False)
mae = mean_absolute_error(y, y_pred)

# Print performance
print("Image-level Model Performance:")
print(f"R² Score: {r2:.4f}")
print(f"RMSE (tons/ha): {rmse:.4f}")
print(f"MAE (tons/ha): {mae:.4f}")


Image-level Model Performance:
R² Score: 0.4063
RMSE (tons/ha): 0.3537
MAE (tons/ha): 0.2865


c:\Users\Dell\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [9]:
# Filter only Munda6 images from the df_indices DataFrame,
# which has already been merged with polygon_areas.csv
df_munda6 = df_indices[df_indices['polygon'] == 'Munda6'].copy()

# Check if Munda6 data is actually present for prediction
if df_munda6.empty:
    print("Error: No Munda6 data found in df_indices for prediction. Ensure it loaded correctly in earlier steps.")
else:
    # Predict yield per hectare for each Munda6 image
    # Only pass the 'features' columns to the model for prediction
    df_munda6['predicted_yield_per_ha'] = model.predict(df_munda6[features])

    # Calculate average predicted yield per hectare across all Munda6 images
    avg_predicted_yield_per_ha = df_munda6['predicted_yield_per_ha'].mean()

    print(f"\nAverage predicted yield per hectare for Munda6: {avg_predicted_yield_per_ha:.3f} tons/ha")


Average predicted yield per hectare for Munda6: 1.287 tons/ha
